In [ ]:
# Estado de la gpu

!nvidia-smi # Hera

Sun Jan 19 22:34:47 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:AF:00.0 Off |                  N/A |
| 41%   27C    P8    15W / 280W |      3MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Instalacion de librerias necesarias para las pruebas

# !pip install pandas scikit-learn numpy xgboost

In [3]:
# Librerías
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import accuracy_score

# Leemos los datos en formato svmlight
X, y = load_svmlight_file('/mnt/homeGPU/joman/TFM-Paralelizacion/data/raw/epsilon_normalized_t') # 1 GPU solo tiene capacidad para almacenar datos de test

# Convertimos las etiquetas a 0 y 1
y = (y + 1) // 2

# Dividimos en entenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# OConvertimos a formato Dmatrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Parámetros para el entrenamiento
params = {
    'objective': 'binary:logistic',   # Clasificacion binaria
    'max_depth': 6,                    # Profundidad
    'learning_rate': 0.1,              
    'eval_metric': 'logloss',          # Función objetivo
    'tree_method': 'hist',         
    'device': 'cuda'                  # Activamos CUDA
}

num_round = 100  # Número de boostings
bst = xgb.train(params, dtrain, num_round)

# Predicciones
y_pred = bst.predict(dtest)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convertimos a 0 y 1

# Mostramos la exactitud del modelo
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8361
